# Goals
- Differentiate between face recognition and verification
- Implement one-shot learning to solve a face recognition problem
- Apply the triplet loss function to learn a network's param in the context of face recognition
- Explain how to pose face recognition as a binary classification problem
- Map face images into 128-dimensional embeddings using a pretrained model
- Perform face verification and recognition with these encodings

# Face Verification

1:1 problem

### Siamese Network

What's the difference between face verification and face recognition

In face verification, given two images, why not just compare pixels by pixels?

Explain how a siamese network work
- what's the loss func
- what's the forward pass
- what's the arch

In [ ]:
import torch
import torch.nn as nn

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(...),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Linear(...),
            nn.Sigmoid(),
        )

    def forward_one(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        return self.fc(x)
    
    def forward(self, x1, x2):
        return self.forward_one(x1), self.forward_one(x2)

> We can have different versions of the SiameseNetwork, where the output is either a binary classification or a pair or embeddings

### Triplet Loss

Explain how do triplet loss help us in our task, what does it do and how does it help us in our task?

In [1]:
import torch

def triplet_loss(y_pred, margin=0.2):
    # Ensure y_pred has the correct shape
    assert y_pred.dim() == 3 and y_pred.shape[1] == 3, "y_pred should have shape (batch_size, 3, embedding_dim)"
    
    anchor, pos, neg = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]
    
    # Compute the (encoding) distances
    pos_dist = torch.sum(torch.pow(anchor - pos, 2), dim=-1)
    neg_dist = torch.sum(torch.pow(anchor - neg, 2), dim=-1)
    
    # Compute the triplet loss
    basic_loss = pos_dist - neg_dist + margin
    
    # Apply hinge loss and take the mean
    return torch.clamp(basic_loss, min=0.0).mean()

### Embedding Images

Here we need to load image and run the forward pass of the model on the image to get the embedding of a given image

### Verify

In [3]:
import torch

def img_to_embedding(img_path, model):
    pass

def verify_identity_against_db(image_path, identity, database, model, margin):
    embedding = img_to_embedding(image_path, model)
    dist = torch.linalg.vector_norm(embedding - database[identity], p=2)

    if dist < margin:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False       
    return dist, door_open


# Face Recognition

In [ ]:
import torch

def img_to_embedding(img_path, model):
    pass

def find_identity_from_db(image_path, identity, database, model, margin):
    embed = img_to_embedding(image_path, model)
    
    min_dist = float('inf')
    for (db_name, db_embed) in database.items():
        # depending on the forward pass of ur model
        dist = torch.linalg.vector_norm(embed - db_embed, p=2)
        if dist < min_dist:
            min_dist = dist
            identity = db_name

    if min_dist > margin:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
    return min_dist, identity


1:K problem

### Papers
- [FaceNet](https://arxiv.org/pdf/1503.03832)
- [DeepNet](https://scontent-cdg4-2.xx.fbcdn.net/v/t39.8562-6/240890413_887772915161178_4705912772854439762_n.pdf?_nc_cat=109&ccb=1-7&_nc_sid=e280be&_nc_ohc=E5zgFzveiqoQ7kNvgEAT5z0&_nc_ht=scontent-cdg4-2.xx&oh=00_AYD2KkfdtwA8HHWJxlYahOKXHTWdCLPuEhk7DbZgA4a_1A&oe=66BBE9FF)